In [2]:
import os
from google import genai
from google.genai import types
import requests
import json
from pydantic import BaseModel
import asyncio
import nest_asyncio 
from crawl4ai import *

# Add this line to allow nested event loops
nest_asyncio.apply()


In [48]:
async def crawl_website(url_webpage):
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url=url_webpage,
        )
        return result.markdown

def run(url_webpage):
    return asyncio.run(crawl_website(url_webpage))

In [40]:
run("https://www.golem.de/")

[INIT].... → Crawl4AI 0.4.247
[FETCH]... ↓ https://www.golem.de/... | Status: True | Time: 0.00s
[COMPLETE] ● https://www.golem.de/... | Status: True | Total: 0.01s


'[ ![Golem.de - IT-News für Profis](https://www.golem.de/staticrl/images/logo-g.png) ](https://www.golem.de/<https:/www.golem.de/>) [ ](https://www.golem.de/<https:/redirect.golem.de/nl.php?id=account_login_cmp>)\n# Willkommen auf Golem.de!\n## Cookies zustimmen\nBesuchen Sie Golem.de wie gewohnt mit Werbung und Tracking, indem Sie der Nutzung aller Cookies zustimmen. Details zum Tracking finden Sie im Privacy Center. \nUm der Nutzung von Golem.de mit Cookies zustimmen zu können, müssen Cookies in Ihrem Browser aktiviert sein. Weitere Informationen finden Sie [hier](https://www.golem.de/<https:/www.golem.de/sonstiges/techinfo.html>). \nDie Zustimmung in einem iFrame ist nicht möglich. [Seite in eigenem Fenster öffnen](https://www.golem.de/<#>). \nDer Zustimmungs-Dialog konnte nicht korrekt geladen werden, eine Zustimmung gilt nur vorläufig. Informationen zur Problem\xadbehandlung finden Sie [hier](https://www.golem.de/<https:/www.golem.de/sonstiges/techinfo.html>). \nDie Möglichkeit zu

In [6]:
# Only run this block for Gemini Developer API
client = genai.Client()

model_name_thinking = "gemini-2.0-flash-thinking-exp-01-21"
model_name = "gemini-2.0-flash-exp"

In [ ]:
response = client.models.generate_content(
    model=model_name_thinking, contents="Is it difficult to find large prime numbers? If yes, why?"
)
print(response.text)

In [ ]:
system_instruction = """You are an AI assistant designed to produce output that is visually appealing and easily readable in a terminal. When formatting your responses, utilize the syntax of the Python `rich` library. This involves using square brackets to enclose formatting tags.
        Here are some examples of how to apply formatting:

        * **Emphasis:** Instead of "This is important", output "[bold]This is important[/]".
        * **Headers/Titles:** Instead of "Section Title:", output "[bold blue]Section Title:[/]".
        * **Warnings:** Instead of "Warning!", output "[bold red]Warning![/]".
        * **Success Messages:** Instead of "Operation successful.", output "[green]Operation successful.[/]".
        * **Lists:** You can use colors for list items like "[cyan]*[/] Item 1".

        Always use the `rich` library's syntax for formatting terminal output to enhance readability."""

response = client.models.generate_content(
    model=model_name_thinking,
    contents="Show me the proof for the euler identity?",
    config=types.GenerateContentConfig(
        system_instruction=system_instruction,
        temperature=0.3,
    ),
)
print(response.text)

In [ ]:
file = client.files.upload(path="a11.text")
response = client.models.generate_content(
    model=model_name, contents=["Summarize this file", file]
)
print(response.text)

In [4]:
def get_google_search_results(query, num_results=10):
    api_key = os.environ.get("SERPER_API_KEY")

    if not api_key:
        print("SERPER_API_KEY not found in environment variables.")
        return

    url = "https://google.serper.dev/search"
    payload = json.dumps({
    "q": query,
    "num": num_results
    })
    headers = {
    'X-API-KEY': api_key,
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response

response = get_google_search_results("LLM test-time compute and training")

In [5]:
# Convert to JSON
json_response = response.json()
print(json_response)

{'searchParameters': {'q': 'LLM test-time compute and training', 'type': 'search', 'engine': 'google'}, 'organic': [{'title': 'Optimizing LLM Test-Time Compute Involves Solving a Meta-RL ...', 'link': 'https://blog.ml.cmu.edu/2025/01/08/optimizing-llm-test-time-compute-involves-solving-a-meta-rl-problem/', 'snippet': 'In this post, we will discuss one such approach: by altering the LLM training objective, we can reuse existing data along with more test-time ...', 'date': 'Jan 8, 2025', 'position': 1}, {'title': 'Test-Time Compute: The Next Frontier in AI Scaling - IKANGAI', 'link': 'https://www.ikangai.com/test-time-compute-the-next-frontier-in-ai-scaling/', 'snippet': 'Major AI labs are exploring test-time compute, where models receive extra processing time during execution to produce better results.', 'date': 'Nov 12, 2024', 'position': 2}, {'title': 'What is Test Time Compute? | CSA', 'link': 'https://cloudsecurityalliance.org/blog/2024/12/13/test-time-compute', 'snippet': 'Test-tim

In [6]:
json_response['organic'][0]

{'title': 'Optimizing LLM Test-Time Compute Involves Solving a Meta-RL ...',
 'link': 'https://blog.ml.cmu.edu/2025/01/08/optimizing-llm-test-time-compute-involves-solving-a-meta-rl-problem/',
 'snippet': 'In this post, we will discuss one such approach: by altering the LLM training objective, we can reuse existing data along with more test-time ...',
 'date': 'Jan 8, 2025',
 'position': 1}

In [26]:
def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return "sunny"


response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="What is the weather like in Boston?",
    config=types.GenerateContentConfig(tools=[get_current_weather]),
)

print(response.text)

/home/dplaia/Projekte/DeepResearchAgent/.venv/lib/python3.13/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


The weather in Boston is sunny.



In [34]:
class CountryInfo(BaseModel):
    name: str
    population: int
    capital: str
    continent: str
    gdp: int
    official_language: str
    total_area_sq_mi: int


response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="Give me information for the United States.",
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=CountryInfo,
    ),
)
print(response.text)

{
"capital": "Washington, D.C.",
"continent": "North America",
"gdp": 25460000000000,
"name": "United States",
"official_language": "English",
"population": 331002651,
"total_area_sq_mi": 3796742
}
